In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
import pandas as pd
import seaborn as sns
import yaml

from loaders import *

In [134]:
def read_yaml_files_to_dataframe(directory):
    rows_list = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith(".yaml"):
            file_path = os.path.join(directory, filename)
            file_number = filename.split('.')[0]

            with open(file_path, 'r') as file:
                content = yaml.safe_load(file)

                instance_data = content['problem']['instance']
                bottom_name = content['problem']['shape']['name']

                if len(bottom_name.split('_')) < 3:
                    try:
                        group_name = bottom_name.split('_')[0]
                        layer_type = bottom_name.split('_')[1]
                        resid_block_num = None
                        mamba_block = None
                    except:
                        group_name = None
                        layer_type = bottom_name
                        resid_block_num = None
                        mamba_block = None
                else:
                    group_name = bottom_name.split('_')[0]
                    resid_block_num = int(bottom_name.split('_')[1])
                    mamba_block = bottom_name.split('_')[2]
                    if len(bottom_name.split('_')) == 4:
                        layer_type = bottom_name.split('_')[3]
                    else:
                        layer_type = bottom_name.split('_')[3] + '_' + bottom_name.split('_')[4]

                row_data = {'File Number': file_number, 'Name': bottom_name, 'Group': group_name, 'Residual Block Number': resid_block_num, 'Mamba Block': mamba_block, 'Layer Type': layer_type}
                row_data.update(instance_data)
                rows_list.append(row_data)

    df = pd.DataFrame(rows_list)
    return df


def parse_stats(file_path):
    # Define the keys we want to extract from the stats section
    keys = [
        "GFLOPs (@1GHz)", "Utilization", "Cycles", "Energy", "EDP(J*cycle)", "Area",
        "mac", "psum_spad", "weights_spad", "ifmap_spad", "shared_glb", "DRAM", "Total"
    ]
    values = {}
    
    # Start reading file when "Summary Stats" is found
    all_lines = []
    with open(file_path, 'r') as file:
        record = False
        record_compute = False
        for line in file:
            all_lines.append(line)
            if "Summary Stats" in line:
                record = True
            if "fJ/Compute" in line:
                record_compute = True
            if record:
                if ":" in line:
                    key, value = line.split(':')
                    key = key.strip()
                    value = value.strip().split()[0]  # Split to ignore units and take the first value
                    if key in keys:
                        values[key] = value
                if "Total" in line and line.startswith("    Total"):
                    record = False
            if record_compute:
                if "=" in line:  # Look for lines with '=' which denotes key-value pairs in fJ/Compute section
                    key, value = line.strip().split('=')
                    key = key.strip()
                    value = value.strip().split()[0]  # Strip and split to ignore spaces and units
                    values[key] = value
                if "Total" in line and line.startswith("    Total"):  # Stop recording after Total
                    break
            if "Computes = " in line:
                values["computes"] = line.split(" = ")[1]

    file_str = "".join(all_lines)
    glb_access_str = "=== shared_glb ===\n    Total scalar accesses                   : "
    glb_access_index = file_str.index(glb_access_str) + len(glb_access_str)
    glb_end_index = file_str.index("\n", glb_access_index)
    values["shared_glb_accesses"] = file_str[glb_access_index:glb_end_index]

    dram_access_str = "=== DRAM ===\n    Total scalar accesses                   : "
    dram_access_index = file_str.index(dram_access_str) + len(dram_access_str)
    dram_end_index = file_str.index("\n", dram_access_index)
    values["dram_accesses"] = file_str[dram_access_index:dram_end_index]
    return values

def read_stats_from_directories(base_directory):
    rows_list = []

    # Iterate over subdirectories
    for subdir in sorted(os.listdir(base_directory)):
        dir_path = os.path.join(base_directory, subdir)
        if os.path.isdir(dir_path):
            stats_file = os.path.join(dir_path, 'timeloop-mapper.stats.txt')
            if os.path.exists(stats_file):
                # Parse the stats file
                stats = parse_stats(stats_file)
                # Add the file number (subdirectory name) to the row
                stats['Layer Number'] = subdir
                rows_list.append(stats)

    # Create DataFrame from list of rows
    df = pd.DataFrame(rows_list)
    return df

def create_merged_df(OUTPUT_DIRECTORY, BASE_FOLDER):
    merged_df = pd.merge(read_stats_from_directories(OUTPUT_DIRECTORY), read_yaml_files_to_dataframe(BASE_FOLDER), left_on='Layer Number', right_on='File Number', how='inner')
    merged_df = merged_df.drop(columns=['File Number'])
    return merged_df

def convert_to_floats(merged_df):
    #try converting every column to floats
    for column in merged_df.columns:
        try:
            merged_df[column] = merged_df[column].astype(float)
        except:
            print('failed to convert column to float', column)
            pass

    merged_df['Utilization'] = merged_df['Utilization'].str.rstrip('%').astype('float')
    merged_df.rename(columns={'Utilization': 'Utilization (%)'}, inplace=True)

    return merged_df


merged_df = create_merged_df(
    #"example_designs/example_designs/eyeriss_like/outputs/mamba_1.4b",
    "example_designs/example_designs/eyeriss_like/outputs/gpt2",
    "example_designs/layer_shapes/mamba_1.4b",
)
merged_df = convert_to_floats(merged_df)

failed to convert column to float Utilization
failed to convert column to float Name
failed to convert column to float Group
failed to convert column to float Mamba Block
failed to convert column to float Layer Type


In [135]:
merged_df['Layer Type'] = merged_df['Layer Type'].replace(r'^[1-9]$|^[1-3]\d$|^4[0-7]$', 'matmul', regex=True)

cols = ['Energy', 'Cycles', 'computes', 'shared_glb_accesses', 'dram_accesses']
#merged_df.loc[merged_df['Layer Type'] == 'matmul', cols] *= 1024

In [137]:
merged_df

,GFLOPs (@1GHz),Utilization (%),Cycles,Energy,EDP(J*cycle),Area,computes,mac,psum_spad,weights_spad,...,M,N,P,Q,R,S,Hdilation,Hstride,Wdilation,Wstride
0,23.99,7.14,6.710886e+07,6463.61,434000.0,0.0,8.053064e+08,207.69,2767.51,1978.95,...,8192.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
1,15.99,4.76,3.355443e+07,2095.28,70300.0,0.0,2.684355e+08,207.69,3004.85,1986.27,...,1.0,1.0,1.0,13.0,1.0,4.0,NaN,NaN,NaN,NaN
2,15.97,4.76,3.276800e+04,3.06,0.1,0.0,2.621440e+05,207.69,2808.92,2088.73,...,160.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
3,15.97,4.76,1.310720e+05,11.46,1.5,0.0,1.048576e+06,207.69,2923.31,2088.73,...,4096.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
4,15.99,4.76,1.342177e+08,7575.74,1020000.0,0.0,1.073742e+09,207.69,2762.24,2000.90,...,16.0,1.0,1.0,1.0,1.0,4096.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,15.97,4.76,3.276800e+04,3.06,0.1,0.0,2.621440e+05,207.69,2808.92,2088.73,...,160.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
141,15.97,4.76,1.310720e+05,11.46,1.5,0.0,1.048576e+06,207.69,2923.31,2088.73,...,4096.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
142,15.99,4.76,1.342177e+08,7575.74,1020000.0,0.0,1.073742e+09,207.69,2762.24,2000.90,...,16.0,1.0,1.0,1.0,1.0,4096.0,1.0,1.0,1.0,1.0
143,16.00,4.76,1.342177e+08,7931.19,1060000.0,0.0,1.073742e+09,207.69,3004.85,1986.27,...,2048.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


In [136]:
print("Total energy (mJ):", merged_df["Energy"].sum() / 1000)
print(f"Latency (ms): {merged_df['Cycles'].sum() / 1e9 * 1000}")
print("Total computes (B):", merged_df["computes"].sum() / 1e9)
print("GLB accesses (B):", merged_df["shared_glb_accesses"].sum() / 1e9)
print("DRAM accesses (B):", merged_df["dram_accesses"].sum() / 1e9)

Total energy (mJ): 720.8810699999999
Latency (ms): 10509.123583999999
Total computes (B): 90.515439616
GLB accesses (B): 29.598740224
DRAM accesses (B): 1.667584256


## Misc

In [6]:
show_config('example_designs/layer_shapes/mamba/000.yaml')

problem:
  instance:
    C: 1024
    G: 1
    M: 4096
    N: 1
    P: 1
    Q: 1
    R: 1
    S: 1
  shape:
    coefficients:
    - default: 1024
      name: Cgroup
    - default: 4096
      name: Mgroup
    - default: 1
      name: Hstride
    - default: 1
      name: Wstride
    data_spaces:
    - name: Weights
      projection:
      - - - G
      - - - C
      - - - M
      - - - R
      - - - S
    - name: Inputs
      projection:
      - - - N
      - - - G
          - Cgroup
        - - C
      - - - R
        - - P
          - Hstride
      - - - S
        - - Q
          - Wstride
    - name: Outputs
      projection:
      - - - N
      - - - G
          - Mgroup
        - - M
      - - - P
      - - - Q
      read_write: true
    dimensions:
    - G
    - C
    - M
    - R
    - S
    - N
    - P
    - Q
    name: layers_0_mixer_in_proj




In [7]:
show_config('example_designs/example_designs/eyeriss_like/arch.yaml')

architecture:
  # ============================================================
  # Architecture Description
  # ============================================================
  version: 0.4
  nodes: # Top-level is hierarchical
  - !Container # Top-level system
    name: system
  
  - !Component # DRAM main memory
    name: DRAM
    class: DRAM
    attributes:
      type: "LPDDR4"
      width: 64
      datawidth: 8

  - !Container # Eyeriss accelerator
    name: eyeriss
    attributes:
      technology: "32nm"

  - !Component # Global buffer for inputs & outputs
    name: shared_glb
    class: smartbuffer_SRAM
    attributes:
      depth: 16384
      width: 64
      n_banks: 32
      datawidth: 8
      read_bandwidth: 16
      write_bandwidth: 16
    constraints:
      dataspace: {keep: [Inputs, Outputs], bypass: [Weights]}

  - !Container # Each column of PEs produces a different psum row
    name: PE_column
    spatial: {meshX: 14}
    constraints:
      spatial:
        permutation: [N

In [12]:
with open("example_designs/example_designs/eyeriss_like/outputs/mamba/000/timeloop-mapper.stats.txt") as f:
    stats_str = f.read()

print(stats_str)

Buffer and Arithmetic Levels
----------------------------
Level 0
-------
=== mac ===

    SPECS
    -----
    Word bits             : 16
    Instances             : 168 (14*12)
    Compute energy        : 0.21 pJ

    STATS
    -----
    Utilized instances      : 8
    Computes (total)        : 4194304
    Cycles                  : 524288
    Energy (total)          : 871123.39 pJ
    Area (total)            : 32225.76 um^2

Level 1
-------
=== psum_spad ===

    SPECS
    -----
        Technology                      : SRAM
        Size                            : 16
        Word bits                       : 16
        Block size                      : 1
        Cluster size                    : 1
        Instances                       : 168 (14*12)
        Shared bandwidth                : -
        Read bandwidth                  : 2.00
        Write bandwidth                 : 2.00
        Multiple buffering              : 1.00
        Effective size                  : 16
      